# Azure AI Search gamit ang NVIDIA NIM at LlamaIndex Integration

Sa notebook na ito, ipapakita namin kung paano gamitin ang mga AI model ng NVIDIA at LlamaIndex upang makabuo ng isang makapangyarihang Retrieval-Augmented Generation (RAG) pipeline. Gagamitin natin ang mga LLMs at embeddings ng NVIDIA, isasama ito sa Azure AI Search bilang vector store, at gagamitin ang RAG upang mapabuti ang kalidad at kahusayan ng paghahanap.

## Mga Benepisyo
- **Scalability**: Gamitin ang malalaking language models ng NVIDIA at Azure AI Search para sa scalable at mahusay na retrieval.
- **Cost Efficiency**: I-optimize ang paghahanap at retrieval gamit ang epektibong vector storage at hybrid search techniques.
- **High Performance**: Pagsamahin ang makapangyarihang LLMs sa vectorized search para sa mas mabilis at mas tumpak na mga sagot.
- **Quality**: Panatilihin ang mataas na kalidad ng paghahanap sa pamamagitan ng pag-base ng mga sagot ng LLM sa mga kaugnay na dokumentong nakuha.

## Mga Kinakailangan
- 🐍 Python 3.9 o mas mataas
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key para sa access sa LLMs at Embeddings ng NVIDIA sa pamamagitan ng NVIDIA NIM microservices

## Mga Tampok na Saklaw
- ✅ NVIDIA LLM Integration (gagamitin natin ang [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA Embeddings (gagamitin natin ang [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Advanced Retrieval Modes ng Azure AI Search
- ✅ Document Indexing gamit ang LlamaIndex
- ✅ RAG gamit ang Azure AI Search at LlamaIndex na may NVIDIA LLMs

Simulan na natin!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Pag-install at Mga Kinakailangan
Gumawa ng Python environment gamit ang Python bersyon >3.10.

## Pagsisimula!


Upang makapagsimula, kailangan mo ng `NVIDIA_API_KEY` upang magamit ang mga modelo ng NVIDIA AI Foundation:
1) Gumawa ng libreng account sa [NVIDIA](https://build.nvidia.com/explore/discover).
2) I-click ang modelo na nais mong gamitin.
3) Sa ilalim ng Input, piliin ang tab na Python, at i-click ang **Get API Key** at pagkatapos ay i-click ang **Generate Key**.
4) Kopyahin at i-save ang nabuong key bilang NVIDIA_API_KEY. Mula doon, magkakaroon ka ng access sa mga endpoints.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Halimbawa ng RAG gamit ang LLM at Embedding
### 1) I-initialize ang LLM
Ang `llama-index-llms-nvidia`, na kilala rin bilang LLM connector ng NVIDIA, ay nagbibigay-daan sa iyo na kumonekta at mag-generate mula sa mga compatible na modelo na makikita sa NVIDIA API catalog. Tingnan dito para sa listahan ng mga chat completion models: https://build.nvidia.com/search?term=Text-to-Text

Dito gagamitin natin ang **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) I-initialize ang Embedding
Ang `llama-index-embeddings-nvidia`, na kilala rin bilang NVIDIA's Embeddings connector, ay nagbibigay-daan sa iyo na kumonekta at bumuo mula sa mga compatible na modelo na makikita sa NVIDIA API catalog. Pinili namin ang `nvidia/nv-embedqa-e5-v5` bilang embedding model. Tingnan dito para sa listahan ng mga text embedding models: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Gumawa ng Query Engine para magtanong tungkol sa iyong data

Narito ang isang query gamit ang purong vector search sa Azure AI Search at pag-uugnay ng sagot sa aming LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Narito ang isang query gamit ang hybrid search sa Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Pagsusuri ng Vector Search
Ang tugon ng LLM ay tumpak na nailahad ang mga pangunahing pang-ekonomiyang epekto na binanggit sa orihinal na teksto tungkol sa stock market ng Russia. Partikular, sinabi nito na ang stock market ng Russia ay nakaranas ng malaking pagbagsak, nawalan ng 40% ng halaga nito, at ang kalakalan ay sinuspinde dahil sa kasalukuyang sitwasyon. Ang tugon na ito ay tumutugma nang maayos sa impormasyong ibinigay sa orihinal na teksto, na nagpapakita na tama ang pagkilala at pagsasama ng LLM sa mga mahahalagang detalye tungkol sa epekto sa stock market dulot ng mga aksyon ng Russia at mga ipinataw na parusa.

#### Komentaryo sa Source Nodes
Ang mga source nodes ay nagbibigay ng detalyadong ulat tungkol sa mga pang-ekonomiyang epekto na hinarap ng Russia dahil sa mga internasyonal na parusa. Binibigyang-diin ng teksto na ang stock market ng Russia ay nawalan ng 40% ng halaga nito, at ang kalakalan ay sinuspinde. Bukod dito, binanggit din ang iba pang pang-ekonomiyang epekto, tulad ng pagbaba ng halaga ng Ruble at ang mas malawak na pag-iisa ng ekonomiya ng Russia. Epektibong naipahayag ng tugon ng LLM ang mga mahahalagang punto mula sa mga nodes na ito, na nakatuon sa epekto sa stock market gaya ng hiniling sa query.


Ngayon, tingnan natin ang isang query kung saan ang Hybrid Search ay hindi nagbibigay ng maayos na sagot:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hybrid Search: Pagsusuri ng Tugon ng LLM
Ang tugon ng LLM sa halimbawa ng Hybrid Search ay nagpapahiwatig na ang ibinigay na konteksto ay hindi tumutukoy sa eksaktong petsa ng pagsalakay ng Russia sa Ukraine. Ang tugon na ito ay nagpapakita na ang LLM ay gumagamit ng impormasyong makukuha mula sa mga dokumento ng pinagmulan ngunit kinikilala ang kawalan ng tiyak na detalye sa teksto.

Ang tugon ay tama sa pagtukoy na ang konteksto ay binabanggit ang mga kaganapan na may kaugnayan sa agresyon ng Russia ngunit hindi tinutukoy ang eksaktong petsa ng pagsalakay. Ipinapakita nito ang kakayahan ng LLM na maunawaan ang impormasyong ibinigay habang kinikilala ang mga puwang sa nilalaman. Epektibong hinihikayat ng LLM ang gumagamit na maghanap ng panlabas na mapagkukunan o mga tala ng kasaysayan para sa eksaktong petsa, na nagpapakita ng antas ng pag-iingat kapag hindi kumpleto ang impormasyon.

### Pagsusuri ng Source Nodes
Ang mga source nodes sa halimbawa ng Hybrid Search ay naglalaman ng mga sipi mula sa isang talumpati na tumatalakay sa tugon ng U.S. sa mga aksyon ng Russia sa Ukraine. Ang mga nodes na ito ay nagbibigay-diin sa mas malawak na epekto sa geopolitika at sa mga hakbang na ginawa ng U.S. at ng mga kaalyado nito bilang tugon sa pagsalakay, ngunit hindi binabanggit ang tiyak na petsa ng pagsalakay. Ito ay naaayon sa tugon ng LLM, na tama sa pagtukoy na ang konteksto ay kulang sa tiyak na impormasyon tungkol sa petsa.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hybrid w/Reranking: Pagsusuri ng Tugon ng LLM
Sa halimbawa ng Hybrid w/Reranking, nagbibigay ang tugon ng LLM ng karagdagang konteksto sa pamamagitan ng pagbanggit na ang pangyayari ay naganap anim na araw bago ibinigay ang talumpati. Ipinapakita nito na kaya ng LLM na maghinuha ng petsa ng pagsalakay batay sa oras ng talumpati, kahit na kinakailangan pa rin nito ang eksaktong petsa ng talumpati para sa mas tiyak na impormasyon.

Ipinapakita ng tugon na ito ang mas pinahusay na kakayahan na gumamit ng mga pahiwatig mula sa konteksto upang makapagbigay ng mas makabuluhang sagot. Binibigyang-diin nito ang bentahe ng reranking, kung saan maaaring ma-access at ma-prioritize ng LLM ang mas may kaugnayang impormasyon upang makapagbigay ng mas malapit na approximation ng hinahanap na detalye (hal., ang petsa ng pagsalakay).

### Pagsusuri ng Source Nodes
Ang mga source nodes sa halimbawang ito ay naglalaman ng mga reference sa oras ng pagsalakay ng Russia, partikular na binabanggit na ito ay naganap anim na araw bago ang talumpati. Bagama't ang eksaktong petsa ay hindi pa rin tahasang nabanggit, nagbibigay ang mga nodes ng temporal na konteksto na nagpapahintulot sa LLM na makapagbigay ng mas masusing tugon. Ang pagsasama ng detalyeng ito ay nagpapakita kung paano maaaring mapabuti ng reranking ang kakayahan ng LLM na kumuha at maghinuha ng impormasyon mula sa ibinigay na konteksto, na nagreresulta sa mas tiyak at makabuluhang tugon.


**Tandaan:**
Sa notebook na ito, ginamit namin ang NVIDIA NIM microservices mula sa NVIDIA API Catalog.  
Ang mga API sa itaas, `NVIDIA (llms)`, `NVIDIAEmbedding`, at [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). Tandaan, na ang mga API sa itaas ay maaari ring suportahan ang self-hosted microservices.

**Halimbawa:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")
```



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
